### Bitcoin Prediction - Kaggle

In [8]:
#Installs kagglehub for dataset - uncomment if needed
#!pip install kagglehub

# Imports libraries
import os
import requests
import json
import kagglehub
import pandas as pd
from datetime import datetime

In [9]:
# Download latest version of dataset
path = kagglehub.dataset_download("mczielinski/bitcoin-historical-data")

# List Path & files 
print("Dataset downloaded to:", path)
print("Files:", os.listdir(path))

# Load the CSV 
csv_file = os.path.join(path, "btcusd_1-min_data.csv")
data = pd.read_csv(csv_file)

# Prints head to confirm
print(data.head())

Dataset downloaded to: C:\Users\Devin Ferko\.cache\kagglehub\datasets\mczielinski\bitcoin-historical-data\versions\352
Files: ['btcusd_1-min_data.csv']
      Timestamp  Open  High   Low  Close  Volume
0  1.325412e+09  4.58  4.58  4.58   4.58     0.0
1  1.325412e+09  4.58  4.58  4.58   4.58     0.0
2  1.325412e+09  4.58  4.58  4.58   4.58     0.0
3  1.325412e+09  4.58  4.58  4.58   4.58     0.0
4  1.325412e+09  4.58  4.58  4.58   4.58     0.0


In [10]:
# Convert Timestamp to datetime
data.Timestamp = pd.to_datetime(data.Timestamp, unit='s')

# Resampling to daily frequency
data.index = data.Timestamp
data = data.resample('D').mean()

# Resampling to monthly frequency
data_month = data.resample('ME').mean()

# Resampling to annual frequency
data_year = data.resample('YE-DEC').mean()

# Resampling to quarterly frequency
data_Q = data.resample('QE-DEC').mean()

# Prints head to confirm - Uncomment if needed
data.head()
#data_month.head()
#data_year.head()
#data_Q.head()

,Timestamp,Open,High,Low,Close,Volume
Timestamp,,,,,,
2012-01-01,2012-01-01 17:00:00,4.645697,4.645697,4.645697,4.645697,0.011919
2012-01-02,2012-01-02 11:59:30,4.975000,4.975000,4.975000,4.975000,0.007014
2012-01-03,2012-01-03 11:59:30,5.085500,5.085500,5.085500,5.085500,0.074365
2012-01-04,2012-01-04 11:59:30,5.170250,5.170396,5.170250,5.170396,0.074468
2012-01-05,2012-01-05 11:59:30,5.954292,5.954361,5.954292,5.954361,0.048839


##### We want to look at Volumne Weighted Average Price

What does the VWAP tell you?
VWAP is the abbreviation for volume-weighted average price, which is a technical analysis tool that shows the ratio of an asset's price to its total trade volume. It provides traders and investors with a measure of the average price at which a stock is traded over a given period of time.

In [11]:
# Calculates Weighted Value

data['Weighted_Price'] = (data['High'] + data['Low'] + data['Close']) / 3


In [12]:
# Calculates VWAP
vwap = data['Weighted_Price'] / data['Volume']

# Resamples to the specific frequencies
vwap_d = vwap.resample('D').sum()

# Amends to DataFrame
data = vwap_d.to_frame(name='Weighted Price')

# Prints head to confirm - Comment if needed
data.head()

,Weighted Price
Timestamp,
2012-01-01,389.774000
2012-01-02,709.306931
2012-01-03,68.385869
2012-01-04,69.430884
2012-01-05,121.916678
